#  Tutorial of Applied Cryptography in Python

#### Guevara Noubir, Amirali Sanatinia
##### Northeastern University


A great way to consolidate your understanding of cryptographic is to used them in practical settings. There are a number of crypto libraries in Python, (cryptography, pycrypto, m2crypto). This tutorial uses the cryptography.io. This library provides high level recipes and low level interfaces to common cryptographic algorithms such as symmetric ciphers, message digests, and key derivation functions. You can download the library from [here](https://cryptography.io/) and follow the instructions. You should be able to install the library using the following command: 

```bash
pip install cryptography
```

Note that you need to have pip installed. To install pip, follow the instructions [here](https://pip.pypa.io/)

*note: Each code block has extra imports, so that blocks would be independent runnable code*

## Cyrptography.io
Cryptography components are divided into different submodules. Following is a list of these submodules (not exhaustive)

* Primitive Crypto Blocks (*cryptography.hazmat*)
 * Message Digest and Hashing algorithms (*cryptography.hazmat.primitives.hashes*)
 * Symmetric encryption algorithms (*cryptography.hazmat.primitives.ciphers*)
 * Asymmetric encryption algorithms (*cryptography.hazmat.primitives.asymmetric*)
* X.509 Ecosystem (*cryptography.x509*)
* Full high level crypto recipe (*cryptography.fernet*)


## Hashing Algorithms

As we discussed in class, the goal is to have a long message as input and produce an output which is much shorter called the hash or message digest. Furthermore, we want it to have properties such as pre-image, second preimage, and  collision resistance. *SHA* is a family of popular hash functions.


### Note on MD5 and SHA-1

MD5 and SHA-1 are considered insecure and **not to be used for security purposes**.

In [13]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
import base64
print("You successfully imported the necessary packages")

You successfully imported the necessary packages


In [4]:
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"A Single Message or")
msg_digest = digest.finalize()
print ("SHA256", len(msg_digest), len(msg_digest) * 8, base64.b64encode(msg_digest))

SHA256 32 256 b'2OMi/1VrbIqEin/Ihxcyo6uKjbBGhcDkNmlqcfWqggY='


#### You can also compute a digest one piece at a time.

In [5]:
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"A Single ")
digest.update(b"Message or")
msg_digest = digest.finalize()# Notice the output size of the digest
print ("SHA256", len(msg_digest), len(msg_digest) * 8, base64.b64encode(msg_digest))

SHA256 32 256 b'2OMi/1VrbIqEin/Ihxcyo6uKjbBGhcDkNmlqcfWqggY='


#### Note that both hashing methods result in the same digest. The advantage of the second method is that you can hash a very long message (e.g., a large file that cannot fit in memory) iteratively.

### SHA Family

Secure Hash Algorithm (SHA) family, is a series of hashing algorithms. Ranging from SHA-0 to SHA-3. SHA-0 should never be used, it's strongly advised to move away from SHA-1 to SHA-2 and SHA-3. Recent research indicates that SHA-1 is insecure. Collision were demonstrated in 2017 (invalidating its use in digital signatures e.g., certificates). SHA-3 is the most recent version, published in 2015.

 * SHA-2: Digest size (224, 256, 384, or 512), Block size (512, 1024)
 * SHA-3: Digest size (224, 256, 384, 512), Block size (1600)

In [6]:
for _hash in [hashes.SHA224, hashes.SHA256, hashes.SHA384, hashes.SHA512, 
              hashes.SHA3_256, hashes.SHA3_384, hashes.SHA3_512]:
    digest = hashes.Hash(_hash(), backend=default_backend())
    digest.update(b"Network")
    digest.update(b"Security")
    msg_digest = digest.finalize()
    # Notice the output size of the digest
    print (_hash.name, len(msg_digest), len(msg_digest) * 8)
    print (base64.b64encode(msg_digest))

sha224 28 224
b'WYm+UTU7ZMq4BAaX1+fwHRBtgt2KRE/ci+J1Zw=='
sha256 32 256
b'Wr+fCs2Rfq5Q9nUtnl1pV8tNxrzYIhzR+8TPdyWMLjg='
sha384 48 384
b'I121lIgEBkunsrznkTk8wvV1rYJT27CxN5w3vND36C9E+bUnfp/carfxDdgi6b0n'
sha512 64 512
b'NRJrp7SwnPeOgak1bffPaKtUfoboqWsmNtp/ybIyhgEtggqeuGvW+uZ5TBcTbQNDnlihqxKnHGtnFuDgN4iSEw=='
sha3-256 32 256
b'pCuIGHpBySzVrR8YMJZJ7au5LRy2HffNi7ND3iiJvp0='
sha3-384 48 384
b'HPxG7JCVo6eet418Ic+XYoV0w+IHWVaL8VJO6BKOOxRGJs8oT6+cm6SeSsVGFPlg'
sha3-512 64 512
b'Wg49aN2xp9t0mkeQAqAikPfuQ/3mnEA4XpoApkxnnbYim1aDacHhZpMi89ZH4ocf73WMr0cKMpPoVNNA2yDkZw=='


In [7]:
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"Network")
digest.update(b"Security")
msg_digest = digest.finalize()

In [8]:
print (base64.b64encode(msg_digest))

b'Wr+fCs2Rfq5Q9nUtnl1pV8tNxrzYIhzR+8TPdyWMLjg='


In [9]:
digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
digest.update(b"NetworkSecurity")
msg_digest = digest.finalize()

In [10]:
print (base64.b64encode(msg_digest))

b'Wr+fCs2Rfq5Q9nUtnl1pV8tNxrzYIhzR+8TPdyWMLjg='


### Hash-based message authentication code (HMAC)

HMAC is used for message authentications combined with a secret key. It provides integrity check and authentication.

<img src="include/SHAhmac.png">

image source: wikipedia

In [11]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, hmac
import os
for _hash in [hashes.SHA224, hashes.SHA256, hashes.SHA384, hashes.SHA512]:
    h = hmac.HMAC(os.urandom(16), _hash(), backend=default_backend())
    h.update(b"Network Security")
    msg_mac = h.finalize()
    print (_hash.name, len(msg_mac), len(msg_mac) * 8, base64.b64encode(msg_mac))

sha224 28 224 b'XylYXsqFFBgSzfhQ5Wm/NXSEmPIzs2OXOwJq8g=='
sha256 32 256 b'/fDakfamw90yjgwHYrQs8tSbehMHcE6H7dM9tXFT8s0='
sha384 48 384 b'wWDThB7kIOMNbWOr6og3L8RHPo6TrDu4bUB9iXW1wGJAqp8TJDYNZtO/5sZdOkHQ'
sha512 64 512 b'mYTaec9P/eVC/SN5fobLSvjkMwq5BWUdwS0khFoXyomCr6kk4bPIdrfd/ALlk3Dpm90T/xRpZX/iX5hwX6XqEQ=='


## Symmetric Encryption

In the following we look at the symmetric encryption algorithms. In symmetric crpto, we use the same key for encryption and decryption. Therefore, the two parties needs to establish a secret key between them. It's up to 1000 times faster than asymmetric encryption.


### Advanced Encryption Algorithm (AES)

AES is based on Rijndael encryption algorithm, designed by Joan Daemen and Vincent Rijmen. It was one of the algorithms submitted to U.S. National Institute of Standards and Technology (NIST) to replace DES and 3DES. It was published in 1998 and accepted and standardized in 2001.

 * AES supports key sizes of 128/192/256 bits
 * Block size: 128 bit
 * It's iterative rather than Feistel cipher
 * Treats data in 4 groups of 4 bytes
 * Operates on an entire block in every round
 * Resistant against known attacks
 * Speed and code compactness on many CPUs
 * Rijndael block and key size vary between 128, 192, 256
 * However, in AES block size in 128
 * Number of rounds a function of key size
  * 128 bits     10 rounds
  * 192 bits     12 rounds
  * 256 bits     14 rounds

 * Today most implementations use the CPU support (Intel AES-NI)

### Block cipher mode of operation

To encrypt messages of arbitrary size with block ciphers, we use the following algorithms, called the modes of operation. They define how to encrypt each block of the plaintext to produce the corresponding cipher text block. Some of these are complemetly insecure (ECB) and should not be used.

 * Electronic Codebook (ECB)
 * Cipher Block Chaining (CBC)
 * Cipher Feedback (CFB)
 * Output Feedback (OFB)
 * Counter (CTR)
 * Galois Counter Mode (GCM)
 
 
### Electronic Codebook (ECB)

<img src="include/ECB_enc.png">
<img src="include/ECB_dec.png">



### Cipher Block Chaining (CBC)

<img src="include/CBC_enc.png">
<img src="include/CBC_dec.png">



### Counter (CTR)

<img src="include/CTR_enc.png">
<img src="include/CTR_dec.png">

image source: wikipedia

The following images are encrypted with ECB. Note that you can see the pattern in the data. Therefore, ECB is not secure or recommended to be used.

<img src="include/tux.png">   <img src="include/ECB1.png">   <img src="include/ECB2.png">

In [12]:
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
key = os.urandom(16) # in bytes, 128 bits
iv = os.urandom(16)

In [13]:
# ECB Mode, we only need a key
### *** DO NOT ECB. IT IS INSECURE *** ###

cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
# note that we don't need padding here, since len("Network Security") = 16
cipher_text = encryptor.update(b"Network Security1234567890123456") + encryptor.finalize()
print (base64.b64encode(cipher_text))

b'Nl2Oe0/11fDQb5SU82FdkPpRRb89QFrVP7KdgP48c0Y='


###### cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
ct1 = encryptor.update("Network Security1234567890123457")
encryptor.finalize()
print base64.b64encode(ct1)

In [14]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

b'Network Security1234567890123456'

In [15]:
# padding
# first import padding 
from cryptography.hazmat.primitives import padding

padder = padding.PKCS7(128).padder()

# pad the plaintext
padded_data = padder.update(b"Network Security: -")
padded_data += padder.finalize()

print (padded_data)

# encrypt 
cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
encryptor = cipher.encryptor()
cipher_text = encryptor.update(padded_data) + encryptor.finalize()
print (base64.b64encode(cipher_text))

# decrypt
decryptor = cipher.decryptor()
padded_text = decryptor.update(cipher_text) + decryptor.finalize()

#unpad
unpadder = padding.PKCS7(128).unpadder()
plain_text = unpadder.update(padded_text) + unpadder.finalize()
print (plain_text)

b'Network Security: -\r\r\r\r\r\r\r\r\r\r\r\r\r'
b'Nl2Oe0/11fDQb5SU82FdkJA+dVso3dh31RUv28IvNQY='
b'Network Security: -'


In [16]:
# CBC Mode, we also need an IV
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
# note that we don't need padding here, since len("Network Security") = 16
cipher_text = encryptor.update(b"Network Security") + encryptor.finalize()

In [17]:
base64.b64encode(cipher_text)

b'6njU7KYqOvKKnXe7pTOzdQ=='

In [18]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

b'Network Security'

In [19]:
# GCM Mode, we also need an IV
cipher = Cipher(algorithms.AES(key), modes.GCM(iv), backend=default_backend())
encryptor = cipher.encryptor()
# note that we don't need padding here, since len("Network Security") = 16
encryptor.authenticate_additional_data(b"Only authenticate")
cipher_text = encryptor.update(b"Network Security ") + encryptor.finalize()
tag = encryptor.tag

In [20]:
base64.b64encode(cipher_text)

b'1uX0ICwxQH1MoQq+vYcGTGc='

In [21]:
decryptor = Cipher(algorithms.AES(key), modes.GCM(iv,tag), backend=default_backend()).decryptor()
decryptor.authenticate_additional_data(b"Only authenticate")
decrypted_plain_text = decryptor.update(cipher_text) + decryptor.finalize()
print(decrypted_plain_text)

b'Network Security '


In [22]:
# CTR Mode, we don't need padding in CTR mode. In transforms a block cipher into a stream cipher
# we only need to introduce the nonce
cipher = Cipher(algorithms.AES(key), modes.CTR(os.urandom(16)), backend=default_backend())
encryptor = cipher.encryptor()
# len("Network Security CS 6740") = 25, but no padding is needed
cipher_text = encryptor.update(b"Network Security CS(6740)") + encryptor.finalize()

In [23]:
base64.b64encode(cipher_text)

b'RPXVGP2BYGgefM1GulGvG2qtkEXVEi6CuQ=='

In [24]:
decryptor = cipher.decryptor()
decryptor.update(cipher_text) + decryptor.finalize()

b'Network Security CS(6740)'

## Asymmetric Encryption

Asymmetric encryption mechanism use two different keys for encryption and decryption. Therefore, the two parties do not need to share a secret key between them.


### RSA

RSA, is an asymmetric encryption algorithm by Ron Rivest, Adi Shamir, and Leonard Adleman. It was published in 1977. It's security is inspired by the difficulty of factoring. It's hardness assumption is defined as the RSA problem. RSA is significantly slower that symmetric encryption algorithms, and is not used for encrypting large amounts of data. It is mostly used to encrypt the symmetric key that is used for encryption. The textbook RSA is as follows, but the secure way to encrypt a message requires additional protection e.g., using the OAEP mode.


 * $p, q$: two large prime numbers (private, chosen)
 * $n = pq, \phi(n) = (p-1)(q-1)$   (public, calculated)
 * $e$, such that $\gcd(\phi(n), e) = 1,  1 < e < \phi(n)$	(public, chosen)
 * $d = e-1 mod \phi(n)$	(private, calculated)
 * $E(M) = M^e \mod n$
 * $D(M) = M^d \mod n$
 * $D(E(M)) = M^{ed} \mod n = M$
 
 


In [25]:
# Generate a 4096 bit private key
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import rsa

private_key = rsa.generate_private_key(
    public_exponent=65537,
    key_size=4096,
    backend=default_backend())
# to get the public key
public_key = private_key.public_key()

<img src="include/RSA_OAEP.png">

image souce: wikipedia

In [26]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

message = b"The SECRET MESSAGE"
ciphertext = public_key.encrypt(
    message,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA512()),
        algorithm=hashes.SHA512(),
        label=None))
base64.b64encode(ciphertext)

b'BGVQo0XauJYc/2Qb1hW+EGUhlAp7uLKk/wwG2b75sUoevhrTAqrFVRo30jVcMK9MK6MIEON1ggU+RM40B2VKQGk9UtNgBPvi1/UzF5J0W/XelEaDlvzGskNCvasBEu9657GF97Ke+hq1US3LyzmFPpXmELNWDa1Zi8w+74TOmiCwK9CVYm3dk5+spbmTm24bkQX42eBq9qwM22l7wxtpb39xp++2b+MeU1EbABGCqnhAh9TKNFxn61ERoOTgRf/3mDLlA6v0dMyDXlz+NcK1m0HT3OkvgASeOtX5kGq3ka3+VtElmxOr7y7cZIs9XU4uhVIJRbEeUwmY84wdSwvNOEc//Pp3S3gpCsIGb/20R2rdY9Q1KQ6MZlxxKrlv6P6TelPSsqKgrJI6YOgVyOwTFfAEDtB71YyCCBxiM274f6wcb57u5kKktbZ243s99FcP12ASblNC04VEW8fqKWXDVKqIlxUbWz3BQV7VHRN7n0qotJp4mBypPjhoqVR5sEDlvFWVFGkQMxhjJTbqHarFet5qUXtnAOEOD1mtwUojr1Bo+BMMTunLZtg6RWqsQrqJtpPqLTg001DCleBOuiY7BFhUVyhkHN9lYDGTuFNmRS83ecp5hvwsD56yNA4xAJfkdcz/qu6WLqM0NuvakuB5I7EQYxe5A4OBwx2qT9YHIA8='

In [27]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding

plaintext = private_key.decrypt(
    ciphertext,
    padding.OAEP(
        mgf=padding.MGF1(algorithm=hashes.SHA512()),
        algorithm=hashes.SHA512(),
        label=None))
plaintext

b'The SECRET MESSAGE'

In [28]:
plaintext == message

True

### RSA using OpenSSL

To generate keys, use the following instructions:

```bash
 openssl genrsa -out private_key.pem 2048
 openssl pkcs8 -topk8 -inform PEM -outform DER -in private_key.pem -out private_key.der -nocrypt
 openssl rsa -in private_key.pem -pubout -outform DER -out public_key.der
 ```
 

In [29]:
%%bash
openssl genrsa -out private_key.pem 2048
openssl pkcs8 -topk8 -inform PEM -outform DER -in private_key.pem -out private_key.der -nocrypt
openssl rsa -in private_key.pem -pubout -outform DER -out public_key.der

writing RSA key


In [32]:
# import key from a file. E.g., previously generated by OpenSSL
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

with open("private_key.pem", "rb") as key_file:
     private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,
            backend=default_backend())
public_key = private_key.public_key()

In [33]:
# import key from a file. E.g., previously generated by OpenSSL
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization

with open("private_key.der", "rb") as key_file:
     private_key = serialization.load_der_private_key(
            key_file.read(),
            password=None,
            backend=default_backend())
public_key = private_key.public_key()

### Diffie-Hellman Key Exchange
 * https://cryptography.io/en/latest/hazmat/primitives/asymmetric/dh/

In [54]:
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import dh
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

In [44]:
# One can use RFC 3526 to get some commonly used primes for DH
P_1536 = int(
    "FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD1"
    "29024E088A67CC74020BBEA63B139B22514A08798E3404DD"
    "EF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245"
    "E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7ED"
    "EE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3D"
    "C2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F"
    "83655D23DCA3AD961C62F356208552BB9ED529077096966D"
    "670C354E4ABC9804F1746C08CA237327FFFFFFFFFFFFFFFF",
    16,
)

P_4096 = int (
      "FFFFFFFFFFFFFFFFC90FDAA22168C234C4C6628B80DC1CD1"
"29024E088A67CC74020BBEA63B139B22514A08798E3404DD"
"EF9519B3CD3A431B302B0A6DF25F14374FE1356D6D51C245"
"E485B576625E7EC6F44C42E9A637ED6B0BFF5CB6F406B7ED"
"EE386BFB5A899FA5AE9F24117C4B1FE649286651ECE45B3D"
"C2007CB8A163BF0598DA48361C55D39A69163FA8FD24CF5F"
"83655D23DCA3AD961C62F356208552BB9ED529077096966D"
"670C354E4ABC9804F1746C08CA18217C32905E462E36CE3B"
"E39E772C180E86039B2783A2EC07A28FB5C55DF06F4C52C9"
"DE2BCBF6955817183995497CEA956AE515D2261898FA0510"
"15728E5A8AAAC42DAD33170D04507A33A85521ABDF1CBA64"
"ECFB850458DBEF0A8AEA71575D060C7DB3970F85A6E1E4C7"
"ABF5AE8CDB0933D71E8C94E04A25619DCEE3D2261AD2EE6B"
"F12FFA06D98A0864D87602733EC86A64521F2B18177B200C"
"BBE117577A615D6C770988C0BAD946E208E24FA074E5AB31"
"43DB5BFCE0FD108E4B82D120A92108011A723C12A787E6D7"
"88719A10BDBA5B2699C327186AF4E23C1A946834B6150BDA"
"2583E9CA2AD44CE8DBBBC2DB04DE8EF92E8EFC141FBECAA6"
"287C59474E6BC05D99B2964FA090C3A2233BA186515BE7ED"
"1F612970CEE2D7AFB81BDD762170481CD0069127D5B05AA9"
"93B4EA988D8FDDC186FFB7DC90A6C08F4DF435C934063199"
"FFFFFFFFFFFFFFFF",16)

pn = dh.DHParameterNumbers(P_4096,2)
parameters = pn.parameters()
print(parameters.parameter_numbers().g, parameters.parameter_numbers().p)

2 10443888814131525066796027198465295458312690609921350090225887564443381720223226907104440466698097839301115857378903626918601270792704954545172186730169284274591460018668857797629822293211923683033462352043680510103091556741556974603471769463940765351572849948952848216337009218117167389724518349794558970103063334685907513583651387822503722691179689851943224445356874155220071516386381414561784206212778226749950279902786734586295443917369197662990055115054461776681544462348826659616807965769031991160893476349471877789065280080047566925716669229641225661745827767073324523710012721637768412293183249031257407135741410051245619659138888997534617353479700116932563167516606789508300275102558048461055834650554466150904443095830507758085092970400396800574353422539265662408981958636315888889363641299200593084556694540340103914782387841898885946723362427637951381763532228455246440400942589624336133540361046438819252384892240101941930889116661655842294246681654416889277904606082648642042377170020

In [ ]:
# Alternatively one can generate the DH parameters automatically
## This can take a long time
parameters = dh.generate_parameters(generator=2, key_size=4096)

In [45]:
print(parameters.parameter_numbers().g, parameters.parameter_numbers().p)

2 10443888814131525066796027198465295458312690609921350090225887564443381720223226907104440466698097839301115857378903626918601270792704954545172186730169284274591460018668857797629822293211923683033462352043680510103091556741556974603471769463940765351572849948952848216337009218117167389724518349794558970103063334685907513583651387822503722691179689851943224445356874155220071516386381414561784206212778226749950279902786734586295443917369197662990055115054461776681544462348826659616807965769031991160893476349471877789065280080047566925716669229641225661745827767073324523710012721637768412293183249031257407135741410051245619659138888997534617353479700116932563167516606789508300275102558048461055834650554466150904443095830507758085092970400396800574353422539265662408981958636315888889363641299200593084556694540340103914782387841898885946723362427637951381763532228455246440400942589624336133540361046438819252384892240101941930889116661655842294246681654416889277904606082648642042377170020

In [46]:
# Generate a private key for A for use in the exchange.
private_key = parameters.generate_private_key()

In [47]:
# Typically the peer public key is received from peer. Here we will generate it ourselves.
peer_public_key = parameters.generate_private_key().public_key()


In [48]:
# Compute the DH key
shared_key = private_key.exchange(peer_public_key)

# Perform key derivation.
derived_key = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key)

In [52]:
# For the next handshake we MUST generate another private key, but
# we can reuse the parameters.
private_key_2 = parameters.generate_private_key()
peer_public_key_2 = parameters.generate_private_key().public_key()
shared_key_2 = private_key_2.exchange(peer_public_key_2)
derived_key_2 = HKDF(
    algorithm=hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
).derive(shared_key_2)

print(base64.b64encode(derived_key_2))

b'j3U5Ejw7doo6Dcr3cmpXi1w07rZPfi4g4U26GC2OhKc='


### Elliptic Curve Cryptography: Signatures
 * Ed25519 is an elliptic curve signing algorithm using EdDSA and Curve25519

In [58]:
from cryptography.hazmat.primitives.asymmetric.ed25519 import Ed25519PrivateKey
private_key = Ed25519PrivateKey.generate()
signature = private_key.sign(b"my authenticated message")
public_key = private_key.public_key()
# Raises InvalidSignature if verification fails
public_key.verify(signature, b"my authenticated message")

None


## Using JSON

In [30]:
# You can load and dump json objects
import json
person = '{"name": "", "languages": ["English", "French"]}'
person_dict = json.loads(person)
person_dict["name"] = "GN" 
j = json.dumps(person_dict)
print(j)
l = json.loads(j)
print(l["name"])

# You can set fields to bytes object after encoding them for instance using base64
n = base64.b64encode(b"0177")
person_dict["name"] = n.decode()
j = json.dumps(person_dict)
print(j)
l = json.loads(j)
print(base64.b64decode(l["name"]))



{"name": "GN", "languages": ["English", "French"]}
GN
{"name": "MDE3Nw==", "languages": ["English", "French"]}
b'0177'
